## Passing Authority: Multisig accounts and Rekeying
#### 03.5 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-02

See also: https://developer.algorand.org/tutorials/creating-python-transaction-purestake-api/

- Load credentials
- Create our own QR code for payments
- Interact with the blockchain and execute a payment from Python

## Setup
Starting with chapter 3.4, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import PaymentTxn, MultisigTransaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [ ]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

## Multisig account
Alice, Bob and Charlie create a multisig account, 
with the requirement that at least 2 out of 3 have to sign.

#### Step 1: Create the Multisig account
* Using the new command `Multisig`

In [ ]:
from algosdk.future.transaction import Multisig

# create a multisig account
version   = 1     # multisig version, currently always 1
threshold = 2     # min 2 signatures required

account_list = [Alice['public'], Bob['public'], Charlie['public']]

msig = Multisig(version, threshold, account_list)
msig_address = msig.address()

print("Multisig Address: ", msig_address)
print('https://testnet.algoexplorer.io/address/'+msig_address)

#### Step 2: Fund the multisig account
* Can be funded via PaymentTxn
* Here we fund from (yet another) dispenser for free ALGOs

In [ ]:
print('https://dispenser.testnet.aws.algodev.network/?account='+msig_address)

#### Step 3: Make a Multisig payment transaction
* Together, they want to send 1.2 ALGOs to Dina
* Alice and Bob sign the transaction

In [ ]:
sp = algod_client.suggested_params()
amt = int(1.2*1E6)

txn = PaymentTxn(sender=msig_address,         # <-- From Multisig address
                 sp=sp, 
                 receiver=Dina['public'], 
                 amt = amt
                )

#### Step 4: Create a multisig object and sign it

In [ ]:
# Create multisig object
mtx = MultisigTransaction(txn, msig)

# Collect at least 2 signatures
mtx.sign(Alice['private'])
mtx.sign(Bob['private'])

#### Step 5: Send the transaction

In [ ]:
txid = algod_client.send_transaction(mtx)
print("Send transaction with txID: {}".format(txid))

#### Step 6: Wait for confirmation

In [ ]:
txinfo = wait_for_confirmation(algod_client, txid)

#### Check Dina's holdings

In [ ]:
print('https://testnet.algoexplorer.io/address/'+Dina['public'])

#### Exercise
* Repeat steps 3-6 with Alice and Charlie signing the transaction
* Repeat steps 3-6 with Alice, Bob and Charlie signing 
* Repeat steps 3-6 with only Alice signing

## Rekeying
Case 1
* Imagine your private key has been compromised
* You have lots of assets and maybe ongoing smart contracts with this address
* So you want to "change the password"

Case 2
* Bob is very busy and wants Charlie to sign for him

#### Bob rekeys to Charlie
Rekeying is done via a simple payment transaction of 0 ALGOs, that has an additional rekey entry

In [ ]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Bob does not want to send any ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <-- The address which is rekeyed
                 sp = sp, 
                 receiver = Bob['public'],       # <-- The address which is rekeyed 
                 amt = amt,
                 rekey_to = Charlie['public']    # <-- Charlie should be able to sign for Bob
                )

In [ ]:
# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob signs one last time
txid = algod_client.send_transaction(stxn)

In [ ]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Charlie signs for Bob
* Charlie makes a payment of 1.4 ALGO to Alice out of Bob's account

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(1.4*1E6)                                

txn = PaymentTxn(sender = Bob['public'],           # <-- From Bob
                 sp = sp, 
                 receiver = Alice['public'],       # <-- To Alice 
                 amt = amt
                 )

# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])                # <-- Charlie now signs for Bob
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Bob cannot sign for himself
* Bob wants to make a payment of 1.7 ALGO to Dina

In [ ]:
# Step 1: prepare special transaction

sp = algod_client.suggested_params()
amt = int(1.7*1E6)                                

txn = PaymentTxn(sender = Bob['public'],         # <-- From Bob
                 sp = sp, 
                 receiver = Bob['public'],       # <-- To Dina 
                 amt = amt
                )

# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob tries to sign
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Charlie rekeys back to Bob
* Charlie rekeys the Bob account back to Bob
* Important not to mess up the rest of the course material

In [ ]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Rekeying = send 0 ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <-- The address which is rekeyed
                 sp = sp, 
                 receiver = Bob['public'],       # <-- The address which is rekeyed 
                 amt = amt,
                 rekey_to = Bob['public']        # <-- Bob should (again) be able to sign for himself
                )

In [ ]:
# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])              # <-- Charlie signs the rekeying
txid = algod_client.send_transaction(stxn)

In [ ]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Test the rekeying
* To test the rekeying, run the payment transactions above again
    * Charlie cannot sign for Bob anymore
    * Bob can sign again for himself

## Things that can and will go wrong
* Be sure if you trust the new signature holder
* Wrong address or typo in rekey means you loose all your money
    * The protocol does not check whether you or someone has actually access to the rekey address